In [13]:
import os
import re
import pandas as pd

def parse_best_so_far(file_path):
    """
    Parse the best-so-far configurations from a single file.
    """
    best_so_far = []
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Regex para capturar as configurações best-so-far
    iteration_blocks = re.findall(
        r"Iteration \d+ of \d+.*?Best-so-far configuration:\s*(\d+).*?mean value:\s*([\d.]+).*?Description of the best-so-far configuration:\s*((?:.|\n)*?)#",
        content, re.MULTILINE | re.DOTALL
    )
    
    for config_id, mean_value, description in iteration_blocks:
        # Processar a descrição para extrair os parâmetros
        params = {}
        description_lines = description.strip().split("\n")
        
        for i in range(0, len(description_lines), 2):
            if i + 1 < len(description_lines):
                keys = description_lines[i].strip().split()
                values = description_lines[i + 1].strip().split()
                for key, value in zip(keys, values):
                    params[key] = value
        
        # Adicionar o resultado no formato desejado
        best_so_far.append({
            "config_id": config_id,
            "mean_value": float(mean_value),
            "parameters": params,
            "name": os.path.basename(file_path).split(".")[0].split("_")[1]
        })
    
    menor_dicionario = min(best_so_far, key=lambda d: d["mean_value"])
    # # Adicionar a chave "name" com o nome base do arquivo
    # for entry in best_so_far:
    #     entry["name"] = os.path.basename(file_path).split(".")[0].split("_")[1]
    
    
    return menor_dicionario

def parse_directory(directory_path):
    """
    Parse all files in a directory for best-so-far configurations.
    """
    results = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith(".txt"):
            results.append(parse_best_so_far(file_path))

    # Transformando a lista de dicionários em um DataFrame
    df = pd.DataFrame([{**{"name": d["name"]}, **d["parameters"]} for d in results])
    
    return df



In [14]:
import subprocess
import os
import pandas as pd

def run_hgs(df, conjunto_name, path_inst):
    # Defina os parâmetros que deseja passar
    parametros = ['nbIterTraces', 'nbGranular', 'mu', 'lambda', 'nbElite', 'nbClose', 'nbIterPenaltyManagement', 'targetFeasible', 'penaltyIncrease', 'penaltyDecrease' ]  # Modifique conforme necessário

    valores = dict(df.loc[df["name"] == conjunto_name, parametros])

    nbIterTraces = valores['nbIterTraces'].item()
    nbGranular = valores['nbGranular'].item()
    mu = valores['mu'].item()
    lambda_ = valores['lambda'].item()  # 'lambda' é reservado, por isso usamos 'lambda_'
    nbElite = valores['nbElite'].item()
    nbClose = valores['nbClose'].item()
    nbIterPenaltyManagement = valores['nbIterPenaltyManagement'].item()
    targetFeasible = valores['targetFeasible'].item()
    penaltyIncrease = valores['penaltyIncrease'].item()
    penaltyDecrease = valores['penaltyDecrease'].item()


    comando = f'../build/hgs {path_inst} mySolution_test.sol -log 0 -seed 1 -t 20 -nbIterTraces {nbIterTraces} -nbGranular {nbGranular} -mu {mu} -lambda {lambda_} -nbElite {nbElite} -nbClose {nbClose} -nbIterPenaltyManagement {nbIterPenaltyManagement} -targetFeasible {targetFeasible} -penaltyIncrease {penaltyIncrease} -penaltyDecrease {penaltyDecrease}'

    processo = subprocess.run(comando, shell=True, capture_output=True, text=True)

    print("stdout: ", processo)
    value_resp = int(processo.stdout)
    print(value_resp)
    return value_resp


def run_all(df, diretorio):
    mean_dict  = {}
    # Percorrer todos os arquivos no diretório
    for filename in os.listdir(diretorio):
        if filename.endswith(".txt"):  # Processar apenas arquivos .txt
            caminho_arquivo = os.path.join(diretorio, filename)
            
            # Nome do arquivo sem a extensão .txt
            conjunto_name = os.path.splitext(filename)[0]
            
            print(f"\n### Conjunto de instancias: {conjunto_name} ###")
            soma = []
            # Ler o conteúdo do arquivo e imprimir os caminhos
            with open(caminho_arquivo, "r") as file:
                for linha in file:
                    path = linha.strip()
                    value = run_hgs(df, conjunto_name, path)
                    soma.append(value)
            
                mean_dict[conjunto_name] = (sum(soma))/(len(soma))

    return mean_dict


In [21]:
## Para o J1

directory_path = '/Users/jvmaues/Documents/Estudos/Mestrado/optimization/cvrp-cec/Federated/log_j3'
df_params = parse_directory(directory_path)

df_params.to_csv("params_J3.csv")

In [22]:
df_params

,name,.ID.,nbIterTraces,nbGranular,mu,lambda,nbElite,nbClose,nbIterPenaltyManagement,targetFeasible,penaltyIncrease,penaltyDecrease,.PARENT.
0,K7E7C18J3,1,1,250,25,25,20,5,1,100,0.3,1.2,1
1,K7E6C18J3,9,9,250,25,25,20,5,9,100,0.3,1.3,9
2,K7E5C15J3,8,8,500,20,20,20,7,8,150,0.4,1.4,8
3,K7E4C15J3,8,8,500,15,20,20,7,8,150,0.4,1.2,8
4,K7E2C10J3,4,4,500,20,25,20,7,4,150,0.2,1.3,4
5,K7E3C10J3,2,2,700,20,30,40,7,2,50,0.2,1.2,2
6,K5E1C18J3,1,1,250,25,25,20,5,1,100,0.3,1.2,1
7,K5E4C10J3,3,3,700,20,20,60,5,3,150,0.3,1.3,3
8,K5E5C10J3,3,3,700,20,20,60,5,3,150,0.3,1.3,3
9,K3E1C10J3,4,4,500,20,20,20,5,4,50,0.3,1.2,4


In [23]:
import pandas as pd

# Load the provided CSV file
file_path = 'params_J3.csv'
data = pd.read_csv(file_path)

# Constants
k_values = [3, 5, 7]
c_values = [10, 15, 18]
j_value = 3

# New DataFrame to store results
result_data = []

# Iterate over each k, c, and corresponding rows
for k in k_values:
    for c in c_values:
        # Filter rows based on the identifier pattern
        pattern = f"K{k}E[0-9]+C{c}J{j_value}"
        filtered_rows = data[data['name'].str.match(pattern)]
        
        # Calculate column means if matching rows are found
        if not filtered_rows.empty:
            row_mean = filtered_rows.mean(numeric_only=True)
            identifier = f"K{k}C{c}"
            row_mean['name'] = identifier
            result_data.append(row_mean)

# Create a new DataFrame with the results
result_df = pd.DataFrame(result_data)

# # Drop unnecessary columns
columns_to_drop = ["Unnamed: 0", ".ID.", ".PARENT."]
result_df_cleaned = result_df.drop(columns=columns_to_drop)

# Convert specified columns to integers and floats
int_columns = [
    "nbIterTraces", "nbGranular", "nbElite", "mu", "lambda",
    "nbClose", "nbIterPenaltyManagement", "targetFeasible"
]
float_columns = ["penaltyIncrease", "penaltyDecrease"]

# Ensure no duplicate column definitions
int_columns = list(set(int_columns))

# Convert columns to appropriate data types
result_df[int_columns] = result_df[int_columns].astype(int)
result_df[float_columns] = result_df[float_columns].astype(float)

result_df

,Unnamed: 0,.ID.,nbIterTraces,nbGranular,mu,lambda,nbElite,nbClose,nbIterPenaltyManagement,targetFeasible,penaltyIncrease,penaltyDecrease,.PARENT.,name
0,19.333333,4.666667,4,566,21,21,26,6,4,100,0.300000,1.233333,4.666667,K3C10
1,32.333333,4.666667,4,500,15,23,20,7,4,133,0.366667,1.333333,4.666667,K3C15
2,23.000000,7.333333,7,700,18,26,33,6,7,133,0.300000,1.266667,7.333333,K3C18
3,16.600000,5.400000,5,620,21,22,36,5,5,100,0.260000,1.320000,5.400000,K5C10
4,16.400000,6.400000,6,490,16,22,24,6,6,130,0.400000,1.380000,6.400000,K5C15
5,29.200000,7.000000,7,440,19,22,24,4,7,90,0.320000,1.240000,7.000000,K5C18
6,22.285714,4.285714,4,671,20,23,37,5,4,92,0.257143,1.242857,4.285714,K7C10
7,19.857143,7.000000,7,464,18,24,25,6,7,128,0.271429,1.314286,7.000000,K7C15
8,22.857143,6.428571,6,421,21,23,22,4,6,107,0.285714,1.242857,6.428571,K7C18


In [24]:
print(result_df)

   Unnamed: 0      .ID.  nbIterTraces  nbGranular  mu  lambda  nbElite  \
0   19.333333  4.666667             4         566  21      21       26   
1   32.333333  4.666667             4         500  15      23       20   
2   23.000000  7.333333             7         700  18      26       33   
3   16.600000  5.400000             5         620  21      22       36   
4   16.400000  6.400000             6         490  16      22       24   
5   29.200000  7.000000             7         440  19      22       24   
6   22.285714  4.285714             4         671  20      23       37   
7   19.857143  7.000000             7         464  18      24       25   
8   22.857143  6.428571             6         421  21      23       22   

   nbClose  nbIterPenaltyManagement  targetFeasible  penaltyIncrease  \
0        6                        4             100         0.300000   
1        7                        4             133         0.366667   
2        6                        7        

In [25]:
result_df.to_csv("agg_params_j3.csv")

In [ ]:
# Diretório contendo os arquivos
dir = "/Users/jvmaues/Documents/Estudos/Mestrado/optimization/HGS-CVRP/Federated/J1"  # Substitua pelo caminho correto

resp = run_all(df_params, dir)

In [37]:
import csv

with open("dados.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["conjunto", "media"])  # Cabeçalho
    for chave, valor in resp.items():
        writer.writerow([chave, valor])


In [ ]:
import os

# Diretório contendo os arquivos
diretorio = "/Users/jvmaues/Documents/Estudos/Mestrado/optimization/HGS-CVRP/Federated/"  # Substitua pelo caminho correto

# Percorrer todos os arquivos no diretório
for filename in os.listdir(diretorio):
    if filename.endswith(".txt"):  # Processar apenas arquivos .txt
        caminho_arquivo = os.path.join(diretorio, filename)
        
        # Nome do arquivo sem a extensão .txt
        nome_arquivo = os.path.splitext(filename)[0]
        
        print(f"\n### Caminhos no arquivo: {nome_arquivo} ###")
        
        # Ler o conteúdo do arquivo e imprimir os caminhos
        with open(caminho_arquivo, "r") as file:
            for linha in file:
                print(linha.strip())  # Removendo espaços extras e quebras de linha





In [ ]:
import os

# Diretório contendo os arquivos .txt
diretorio = "/Users/jvmaues/Documents/Estudos/Mestrado/optimization/cvrp-cec/irace_files_HGS/log_j3"

# String a ser acrescentada no final do nome do arquivo
sufixo = "J3"

# Iterar sobre todos os arquivos no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(".txt"):  # Verifica se o arquivo é .txt
        caminho_antigo = os.path.join(diretorio, arquivo)
        # Adicionar o sufixo antes da extensão
        novo_nome = f"{os.path.splitext(arquivo)[0]}{sufixo}.txt"
        caminho_novo = os.path.join(diretorio, novo_nome)
        
        # Renomear o arquivo
        os.rename(caminho_antigo, caminho_novo)

print("Arquivos renomeados com sucesso!")


In [ ]:
import os

# Diretório base onde estão os arquivos .txt
diretorio_base = "/Users/jvmaues/Documents/Estudos/Mestrado/optimization/cvrp-cec/irace_files_HGS/J3"

# String a ser acrescentada ao nome dos arquivos
sufixo = "J3"

# Percorrer diretórios e subdiretórios
for raiz, _, arquivos in os.walk(diretorio_base):
    for arquivo in arquivos:
        if arquivo.endswith(".txt"):  # Verifica se é um arquivo .txt
            caminho_antigo = os.path.join(raiz, arquivo)
            novo_nome = f"{os.path.splitext(arquivo)[0]}{sufixo}.txt"
            caminho_novo = os.path.join(raiz, novo_nome)
            
            # Renomeia o arquivo
            os.rename(caminho_antigo, caminho_novo)

print("Todos os arquivos foram renomeados com sucesso!")


In [3]:
import os
import re
from collections import defaultdict

def parse_best_so_far(file_path):
    """
    Parse the best-so-far configurations from a single file.
    """
    best_so_far = []
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Regex para capturar as configurações best-so-far
    iteration_blocks = re.findall(
        r"Iteration \d+ of \d+.*?Best-so-far configuration:\s*(\d+).*?mean value:\s*([\d.]+).*?Description of the best-so-far configuration:\s*((?:.|\n)*?)#",
        content, re.MULTILINE | re.DOTALL
    )
    
    for config_id, mean_value, description in iteration_blocks:
        # Processar a descrição para extrair os parâmetros
        params = {}
        description_lines = description.strip().split("\n")
        
        for i in range(0, len(description_lines), 2):
            if i + 1 < len(description_lines):
                keys = description_lines[i].strip().split()
                values = description_lines[i + 1].strip().split()
                for key, value in zip(keys, values):
                    params[key] = value
        
        # Adicionar o resultado no formato desejado
        best_so_far.append({
            "config_id": config_id,
            "mean_value": float(mean_value),
            "parameters": params
        })
    
    # Adicionar a chave "name" com o nome base do arquivo
    for entry in best_so_far:
        entry["name"] = os.path.basename(file_path).split(".")[0]
    
    return best_so_far

def parse_directory_with_scenarios(directory_path):
    """
    Parse all files in a directory for best-so-far configurations and include scenario enumeration.
    """
    results = []
    scenarios = defaultdict(list)
    
    # Agrupar arquivos por cenários com base no número após 'K'
    for file_name in os.listdir(directory_path):
        if file_name.endswith(".txt"):
            match = re.match(r"K(\d+)E(\d+)C\d+", file_name)
            if match:
                k_value, e_value = match.groups()
                scenario_id = f"K{k_value}"
                scenarios[scenario_id].append(file_name)
    
    # Processar os arquivos agrupados por cenário
    for scenario_id, files in scenarios.items():
        for file_name in files:
            file_path = os.path.join(directory_path, file_name)
            if os.path.isfile(file_path):
                parsed_data = parse_best_so_far(file_path)
                for entry in parsed_data:
                    entry["scenario"] = scenario_id  # Adiciona o identificador do cenário
                results.extend(parsed_data)
    
    return results

# Exemplo de uso
directory_path = '/Users/jvmaues/Documents/Estudos/Mestrado/optimization/cvrp-cec/irace_files_HGS/log_j1'
results = parse_directory_with_scenarios(directory_path)

# Exibindo os resultados
for result in results:
    print(result)


In [ ]:
import random
import pandas as pd

# Função para gerar dados sintéticos
def generate_synthetic_data():
    scenarios = [
        ["k3E1C10", "k3E2C10", "k3E3C10"],
        ["k3E1C15", "k3E2C15", "k3E3C15"],
        ["k3E1C18", "k3E2C18", "k3E3C18"]
    ]

    synthetic_data = []
    instance_id = 1  # Identificador incremental simples para cada cenário completo

    for scenario_group in scenarios:
        for enterprise_clients in scenario_group:
            for config_id in range(1, 6):  # Máximo de 5 config_id por empresa
                config = {
                    "instance": instance_id,  # ID único para o cenário completo
                    "enterprise_clients": enterprise_clients,  # Ex.: k3E1C10
                    "config_id": f"config_{config_id}",
                    "nbIterTraces": random.randint(500, 1000),
                    "nbGranular": random.randint(10, 30),
                    "mu": random.randint(10, 50),
                    "lambda": random.randint(15, 50),
                    "nbElite": random.randint(2, 10),
                    "nbClose": random.randint(2, 10),
                    "nbIterPenaltyManagement": random.randint(30, 200),
                    "targetFeasible": round(random.uniform(0.1, 0.9), 2),
                    "penaltyIncrease": round(random.uniform(1.0, 2.0), 2),
                    "penaltyDecrease": round(random.uniform(0.5, 0.9), 2)
                }
                synthetic_data.append(config)
        instance_id += 1  # Incrementa apenas após finalizar todas as empresas do mesmo cenário
    
    return synthetic_data

# Gerar dados sintéticos
data = generate_synthetic_data()

# Converter para DataFrame para visualização e manipulação
df = pd.DataFrame(data)

# Salvar os dados gerados para análise futura (opcional)
df.to_csv("synthetic_data.csv", index=False)

# Exibir os dados gerados
df


In [ ]:
import pandas as pd

# Load the provided CSV file
file_path = 'params_J1.csv'
data = pd.read_csv(file_path)

# Constants
k_values = [3, 5, 7]
c_values = [10, 15, 18]
j_value = 1

# New DataFrame to store results
result_data = []

# Iterate over each k, c, and corresponding rows
for k in k_values:
    for c in c_values:
        # Filter rows based on the identifier pattern
        pattern = f"K{k}E[0-9]+C{c}J{j_value}"
        filtered_rows = data[data['name'].str.match(pattern)]
        
        # Calculate column means if matching rows are found
        if not filtered_rows.empty:
            row_mean = filtered_rows.mean(numeric_only=True)
            identifier = f"K{k}C{c}"
            row_mean['name'] = identifier
            result_data.append(row_mean)

# Create a new DataFrame with the results
result_df = pd.DataFrame(result_data)

# Drop the specified columns
columns_to_drop = ["Unnamed: 0", ".ID.", ".PARENT."]
result_df_cleaned = result_df.drop(columns=columns_to_drop)


result_df_cleaned



In [ ]:
import pandas as pd

# Load the provided CSV file
file_path = 'params_J3.csv'
data = pd.read_csv(file_path)

# Constants
k_values = [3, 5, 7]
c_values = [10, 15, 18]
j_value = 1

# New DataFrame to store results
result_data = []

# Iterate over each k, c, and corresponding rows
for k in k_values:
    for c in c_values:
        # Filter rows based on the identifier pattern
        pattern = f"K{k}E[0-9]+C{c}J{j_value}"
        filtered_rows = data[data['name'].str.match(pattern)]
        
        # Calculate column means if matching rows are found
        if not filtered_rows.empty:
            row_mean = filtered_rows.mean(numeric_only=True)
            identifier = f"K{k}C{c}"
            row_mean['name'] = identifier
            result_data.append(row_mean)

# Create a new DataFrame with the results
result_df = pd.DataFrame(result_data)

# # Drop unnecessary columns
columns_to_drop = ["Unnamed: 0", ".ID.", ".PARENT."]
result_df_cleaned = result_df.drop(columns=columns_to_drop)

# Convert specified columns to integers and floats
int_columns = [
    "nbIterTraces", "nbGranular", "nbElite", "mu", "lambda",
    "nbClose", "nbIterPenaltyManagement", "targetFeasible"
]
float_columns = ["penaltyIncrease", "penaltyDecrease"]

# Ensure no duplicate column definitions
int_columns = list(set(int_columns))

# Convert columns to appropriate data types
result_df_cleaned[int_columns] = result_df_cleaned[int_columns].astype(int)
result_df_cleaned[float_columns] = result_df_cleaned[float_columns].astype(float)


